In [ ]:
!pip install pretty_midi
import pretty_midi

     |████████████████████████████████| 5.6MB 2.3MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp36-none-any.whl size=5591954 sha256=cf177719a57b64645b7bd598f6ff55ceb89636f24ab972ed2331fa01aab27b2f
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841db1112c6e5866d75a6b6bf1bef73b874782556ba66
Successfully built pretty-midi


In [35]:
c = 0;

midi_data = pretty_midi.PrettyMIDI("/content/drive/My Drive/Nokia/TPD/jazz/WhyDoILoveYou.mid")
print(midi_data.instruments)
for instrument in midi_data.instruments:
  print(instrument.program)    
  print(instrument.pitch_bends)
  print(instrument.get_piano_roll(1)[1])
  if(instrument.program == 0):
    c = c + 1;
  print(" next instrument")
  print(instrument.is_drum)
  print(len(instrument.notes))
  for i,note in enumerate(instrument.notes):
    print(note)
    if(i > 4):
      break;
print(c)

[Instrument(program=0, is_drum=False, name="")]
0
[]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
 next instrument
False
2139
Note(start=0.025000, end=0.046875, pitch=84, velocity=71)
Note(start=0.028125, end=0.056250, pitch=64, velocity=71)
Note(start=0.021875, end=0.056250, pitch=77, velocity=79)
Note(start=0.018750, end=0.071875, pitch=89, velocity=82)
Note(start=0.053125, end=0.078125, pitch=53, velocity=26)
Note(start=0.028125, end=

In [ ]:
def encode_song(song):
  midi_data = pretty_midi.PrettyMIDI(song)
  encoded_song = []
  for instrument in midi_data.instruments:
    if( instrument.name == "Drum Kit 2"):
      continue;
    if not instrument.is_drum : 
      if(len(instrument.notes) == max_instrument_notelength[final_list.index(song)]):
        for n in instrument.notes:
          encoded_song.append(label_encoder(n))
        break
  return encoded_song
encode_song("/content/drive/My Drive/Nokia/TPD/classical/02. Christe Eleison .mid")

NameError: ignored

In [ ]:
one_encoder = OneHotEncoder()
one_encoder.fit(np.linspace(0 , len(labels) - 1, len(labels)).reshape(-1,-1))

In [ ]:
def prep_song_for_input(x):
  x = one_encoder.transform(np.array(encode_song(x)[:60]).reshape(-1,-1)).toarray()
  final_vec = np.zeros((60,chars))
  if(x.shape[0] >= chars):
    final_vec[:60 , chars] = x[:60 , :chars]
  else:
    final_vec[ : x.shape[0] , :chars] = x;
  return final_vec

In [ ]:
x = []
for i,file in enumerate(final_list):
  try : 
    x.append(prep_song_for_input(file))
  except:
    print(file , i)
x = np.array(x)


In [ ]:
encoder_input = Input(shape = (None , x.shape[2]))

encder_LSTM1 = LSTM(1024 , return_sequences = True)
encoder_outputs = encoder_LSTM1(encoder_input)

encoder_outputs = DropOut(0.3)(encoder_outputs)


encoder_LSTM2 = LSTM(1024 , return_sequences = True)
encoder_outputs , encoder_h , encoder_c = encoder_LSTM2(encoder_outputs)

encoder_states = [encoder_h , encoder_c]

In [ ]:
decoder_input = INput(shape = (None , x.shape[2]))
decoder_LSTM1 = LSTM(1024 , return_sequences = True)

decoder_out = decoder_LSTM1(decoder_input , initial_state = encoder_States)

decoder_LSTM2 = LSTM(1024 , return_sequences = True, return_state = True)
decoder_out ,_,_ = decoder_LSTM2(decoder_out)

decoder_dense = DEnse(x.shape[2] , activation = 'softmax')
decoder_out = decoder_dense(decoder_out)

In [ ]:
model = Model(inputs = [encoder_input , decoder_input] , outputs = [decoder_out])
model.compile(optimizer = 'rmsprop' , loss = 'categorical_crossentropy')
model.fit(x = [x,x] , y = target_data,batch_size = 30 , epochs = 100 , validation_split = 0.2)

In [ ]:


encoder_model = Model(encoder_input , encoder_states)


decoder_state_input_h = Iput(shape(1024 , ))
decoder_state_input_c = Input(shape(1024 , ));

decoder_states_inputs = [decoder_state_input_h , decoder_state_input_c]
decoder_outputs = decoder_LSTM1(decoder_input , initial_state = decider_states_inputs)


decoder_outputs , state_h , state_c = decoder_LSTM2(decoder_outputs)
decoder_states = [state_h , state_c]
decoder_outputs = decoder_dense(decoder_outputs)


decoder_model = Model([decoder_input] + decoder_states_inputs , [decoder_outputs] + decoder_states)


start = np.random.randint(0 , len(x) - 1);
start_value = encoder_model.predict(x[start].reshape(1,60,580));


target_seq = np.zeros((1 , 1 , x.shape[2]))
target_seq[0 , 0 , 0] = 1;
decoded = []

for i in range(60):
  output_tokens , h , c  = decoder_model.predict([target_seq] + states_value)
  sampled_token_index = np.argmax(output_tokens[0 , -1 , :])
  sampled_char = labels[sampled_token_index] = 1;

  states_value = [h,c]

NameError: ignored